In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
EPOCHS = 75
BATCH_SIZE = 128

### Training Data Augmentation

In [3]:
# Training data augmentation
transform_train=transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.2,0.2,0.2)),
])

### Testing Data Normalization

In [4]:
# Normalize the test set same as training set without augmentation
transform_test=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.2,0.2,0.2))
])

### Load training and testing datasets

In [5]:
# Load the training dataset
trainset=torchvision.datasets.CIFAR10(
        root='./data',train=True,download=True,transform=transform_train)

trainloader=torch.utils.data.DataLoader(trainset,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)


Files already downloaded and verified


In [6]:
# Load the testing dataset
testset=torchvision.datasets.CIFAR10(
    root='./data',train=False,download=True,transform=transform_test)
testloader=torch.utils.data.DataLoader(testset,batch_size=BATCH_SIZE,shuffle=False,num_workers=4)

Files already downloaded and verified


In [7]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Define a Convulution Neural Network

In [8]:
import torch
import torch.nn as nn
LR = 0.005

class Net(torch.nn.Module):
    """Some Information about CNN"""

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),  
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        self.conv2 = torch.nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),  
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  
            nn.Dropout(p=0.05,inplace=False)
        )
        self.conv3 = torch.nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),  
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )

        self.conv4 = torch.nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),  
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(p=0.05,inplace=False)
        )

        self.conv5 = torch.nn.Sequential(
            nn.Conv2d(256, 256, 3, padding=1),  
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),  
            nn.Dropout(p=0.05,inplace=False)
            
        )
        
        self.conv6 = torch.nn.Sequential(
            nn.Conv2d(256, 512, 3, padding=1),  
            nn.BatchNorm2d(512),
            nn.ReLU(),
        )
        

        self.gap = nn.AvgPool2d(4,4)
        self.fc = nn.Linear(512, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.gap(x)
        x = x.view(-1, 512)
        x = self.fc(x)
        return x

net = Net()
net.cuda()


Net(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.05, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv4): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3)

### Loss Function and Optimizer

In [9]:
# Define a Loss Function and Optimizer
# I choose to use a Classification Cross-Entropy loss and Adam optimizer

import torch.optim as optim

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(),lr=0.001)

### Training Function

In [10]:
# Tain funcion

def train(model,criterion,optimizer,trainloader,epochs,log_interval=50):
    print('-------- Training job starts ------')
    for epoch in range(epochs):
        running_loss=0.0
        for step,(batch_x,batch_y) in enumerate(trainloader):
            batch_x,batch_y=batch_x.cuda(),batch_y.cuda()
            output=model(batch_x)
            
            optimizer.zero_grad()
            loss=criterion(output,batch_y)
            loss.backward()
            optimizer.step()
            
            running_loss+=loss.item()
            if step % log_interval == (log_interval-1):
                print('[%d, %5d] loss: %.4f' %
                      (epoch + 1, step + 1, running_loss / log_interval))
                running_loss = 0.0
    print('----- Train Finished -----')

### Testing Function

In [11]:
# Test function
def test(model, testloader):
    print('------ Test Start -----')

    correct = 0
    total = 0

    with torch.no_grad():
        for test_x, test_y in testloader:
            images, labels = test_x.cuda(), test_y.cuda()
            output = model(images)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Accuracy of the model is: %.4f %%' % accuracy)
    return accuracy


### Training 

In [12]:
train(net, criterion, optimizer, trainloader, epochs=EPOCHS)


-------- Training job starts ------
[1,    50] loss: 1.7597
[1,   100] loss: 1.4448
[1,   150] loss: 1.3281
[1,   200] loss: 1.2034
[1,   250] loss: 1.0795
[1,   300] loss: 1.0230
[1,   350] loss: 1.0116
[2,    50] loss: 0.8971
[2,   100] loss: 0.8880
[2,   150] loss: 0.8613
[2,   200] loss: 0.8134
[2,   250] loss: 0.7769
[2,   300] loss: 0.7929
[2,   350] loss: 0.7672
[3,    50] loss: 0.6912
[3,   100] loss: 0.7011
[3,   150] loss: 0.6662
[3,   200] loss: 0.6741
[3,   250] loss: 0.6619
[3,   300] loss: 0.6427
[3,   350] loss: 0.6378
[4,    50] loss: 0.6067
[4,   100] loss: 0.6000
[4,   150] loss: 0.5991
[4,   200] loss: 0.5698
[4,   250] loss: 0.6019
[4,   300] loss: 0.5739
[4,   350] loss: 0.5579
[5,    50] loss: 0.5360
[5,   100] loss: 0.5269
[5,   150] loss: 0.5334
[5,   200] loss: 0.5010
[5,   250] loss: 0.5453
[5,   300] loss: 0.5120
[5,   350] loss: 0.5131
[6,    50] loss: 0.4638
[6,   100] loss: 0.4879
[6,   150] loss: 0.4503
[6,   200] loss: 0.4402
[6,   250] loss: 0.4770
[6, 

[48,    50] loss: 0.0569
[48,   100] loss: 0.0596
[48,   150] loss: 0.0580
[48,   200] loss: 0.0662
[48,   250] loss: 0.0711
[48,   300] loss: 0.0671
[48,   350] loss: 0.0665
[49,    50] loss: 0.0552
[49,   100] loss: 0.0630
[49,   150] loss: 0.0548
[49,   200] loss: 0.0624
[49,   250] loss: 0.0582
[49,   300] loss: 0.0625
[49,   350] loss: 0.0628
[50,    50] loss: 0.0608
[50,   100] loss: 0.0584
[50,   150] loss: 0.0524
[50,   200] loss: 0.0588
[50,   250] loss: 0.0594
[50,   300] loss: 0.0565
[50,   350] loss: 0.0707
[51,    50] loss: 0.0470
[51,   100] loss: 0.0567
[51,   150] loss: 0.0569
[51,   200] loss: 0.0583
[51,   250] loss: 0.0613
[51,   300] loss: 0.0627
[51,   350] loss: 0.0649
[52,    50] loss: 0.0547
[52,   100] loss: 0.0533
[52,   150] loss: 0.0516
[52,   200] loss: 0.0546
[52,   250] loss: 0.0608
[52,   300] loss: 0.0651
[52,   350] loss: 0.0531
[53,    50] loss: 0.0530
[53,   100] loss: 0.0586
[53,   150] loss: 0.0527
[53,   200] loss: 0.0519
[53,   250] loss: 0.0564


### Testing

In [13]:
test(net, testloader)

------ Test Start -----
Accuracy of the model is: 87.9000 %


87.9